In [3]:
#from sklearn.neighbors import KNeighborsClassifier
#from sklearn.linear_model import LogisticRegression
#from sklearn import metrics
#import matplotlib.pyplot as plt
#%matplotlib inline
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
data = pd.read_csv("./data/train_users.csv")
data_test = pd.read_csv("./data/test_users.csv")

In [7]:
print "data.shape:" + str(data.shape)
print "data_test.shape:" + str(data_test.shape)

data.shape:(213466, 16)
data_test.shape:(62096, 15)


In [13]:
data[213465:213469]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
213465,axksoy1zyz,2014-09-08,20131203211400,NaN,FEMALE,40,facebook,0,en,seo,facebook,linked,Web,Windows Desktop,Chrome,NDF


In [15]:
data_test.tail()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
62091,cv0na2lf5a,2014-09-30,20140930235232,NaN,-unknown-,31,basic,0,en,direct,direct,untracked,Web,Windows Desktop,IE
62092,zp8xfonng8,2014-09-30,20140930235306,NaN,-unknown-,NaN,basic,23,ko,direct,direct,untracked,Android,Android Phone,-unknown-
62093,fa6260ziny,2014-09-30,20140930235408,NaN,-unknown-,NaN,basic,0,de,direct,direct,linked,Web,Windows Desktop,Firefox
62094,87k0fy4ugm,2014-09-30,20140930235430,NaN,-unknown-,NaN,basic,0,en,sem-brand,google,omg,Web,Mac Desktop,Safari
62095,9uqfg8txu3,2014-09-30,20140930235901,NaN,FEMALE,49,basic,0,en,other,other,tracked-other,Web,Windows Desktop,Chrome


In [3]:
y = data.country_destination.values
data = data.drop(['country_destination'], axis=1)
id_test = data_test['id']
piv_train = data.shape[0]

In [9]:
print "train-test border: %d" % piv_train

train-test border: 213466


In [4]:
#Creating a DataFrame with train+test data
df_all = pd.concat((data, data_test), axis=0, ignore_index=True)
#Removing date_first_booking
df_all = df_all.drop(['date_first_booking'], axis=1)
#Filling nan
#df_all = df_all.fillna(-1)

In [35]:
df_all.columns[70:100]

Index([u'affiliate_provider_craigslist', u'affiliate_provider_daum',
       u'affiliate_provider_direct', u'affiliate_provider_email-marketing',
       u'affiliate_provider_facebook',
       u'affiliate_provider_facebook-open-graph', u'affiliate_provider_google',
       u'affiliate_provider_gsp', u'affiliate_provider_meetup',
       u'affiliate_provider_naver', u'affiliate_provider_other',
       u'affiliate_provider_padmapper', u'affiliate_provider_vast',
       u'affiliate_provider_wayn', u'affiliate_provider_yahoo',
       u'affiliate_provider_yandex', u'first_affiliate_tracked_linked',
       u'first_affiliate_tracked_local ops',
       u'first_affiliate_tracked_marketing', u'first_affiliate_tracked_omg',
       u'first_affiliate_tracked_product',
       u'first_affiliate_tracked_tracked-other',
       u'first_affiliate_tracked_untracked', u'signup_app_Android',
       u'signup_app_Moweb', u'signup_app_Web', u'signup_app_iOS',
       u'first_device_type_Android Phone', u'first_devi

In [8]:
# ALL IDs are UNIQUE!
df_all.id.unique

<bound method Series.unique of 0         gxn3p5htnn
1         820tgsjxq7
2         4ft3gnwmtx
3         bjjt8pjhuk
4         87mebub9p4
5         osr2jwljor
6         lsw9q7uk0j
7         0d01nltbrs
8         a1vcnhxeij
9         6uh8zyj2gn
10        yuuqmid2rp
11        om1ss59ys8
12        k6np330cm1
13        dy3rgx56cu
14        ju3h98ch3w
15        v4d5rl22px
16        2dwbwkx056
17        frhre329au
18        cxlg85pg1r
19        gdka1q5ktd
20        qdubonn3uk
21        qsibmuz9sx
22        80f7dwscrn
23        jha93x042q
24        7i49vnuav6
25        al8bcetz0g
26        bjg0m5otl3
27        hfrl5gle36
28        tp6x3md0n4
29        hql77nu2lk
             ...    
275532    bsv2ev628t
275533    06echc56pl
275534    niqgaye2ov
275535    61iwzuhw6e
275536    gks02el96u
275537    v4r1161l0r
275538    p1clbqd0o6
275539    ozb2z0km6l
275540    w3e3sp6i70
275541    q5bxbq0asg
275542    1xa5t3t0la
275543    zuvz7gfpjz
275544    gpusl6ppgf
275545    gpijioh4eh
275546    3ptlvdxss9
275

In [24]:
df_all.first_affiliate_tracked.value_counts()

untracked        143186
linked            62072
omg               54861
tracked-other      6655
product            2353
marketing           281
local ops            69
Name: first_affiliate_tracked, dtype: int64

### Fix ages

In [10]:
df_all.age.fillna(value=0, inplace=True)
df_all.age[df_all.age > 116] = 0
df_all.age[df_all.age < 15] = 0


/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [11]:
df_all.age.value_counts()

0      117757
30       7713
28       7687
29       7647
31       7542
27       7462
32       7187
33       6796
26       6745
34       6176
25       6150
35       5949
36       5032
24       4571
37       4475
38       4091
39       3642
23       3516
40       3328
41       3098
42       2732
45       2669
44       2585
22       2508
43       2486
46       2317
19       2024
47       2006
48       1779
50       1679
        ...  
102        39
109        36
81         33
85         32
87         31
100        31
86         30
83         30
103        30
107        28
82         28
96         27
84         24
90         21
93         21
106        20
99         19
92         18
98         17
94         17
108        16
89         16
88         15
91         14
97         13
115        13
15          9
113         4
111         2
112         1
Name: age, dtype: int64

### Process date_account_created

In [21]:
#date_account_created
dac = np.vstack(df_all.date_account_created.astype(str).apply(lambda x: list(map(int, x.split('-')))).values)
df_all['dac_year'] = dac[:,0]
df_all['dac_month'] = dac[:,1]
df_all['dac_day'] = dac[:,2]
df_all = df_all.drop(['date_account_created'], axis=1)

### Process timestamp_first_active

In [22]:
#timestamp_first_active
tfa = np.vstack(df_all.timestamp_first_active.astype(str).apply(lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
df_all['tfa_year'] = tfa[:,0]
df_all['tfa_month'] = tfa[:,1]
df_all['tfa_day'] = tfa[:,2]
df_all = df_all.drop(['timestamp_first_active'], axis=1)

### Process One-Hot-Encoding Features

In [25]:
df_all_backup = df_all.copy()

In [26]:
#One-hot-encoding features
ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']
for f in ohe_feats:
    df_all_dummy = pd.get_dummies(df_all[f], prefix=f)
    df_all = df_all.drop([f], axis=1)
    df_all = pd.concat((df_all, df_all_dummy), axis=1)

## Process countries.csv

In [44]:
data_countries = pd.read_csv("./data/countries.csv")
data_age_gender_bkts = pd.read_csv("./data/age_gender_bkts.csv")

In [45]:
# set population_in_thousands
pops = []
for country in data_countries.country_destination.values:
    print "country: %s, popul: %d" % (country, sum(data_age_gender_bkts[data_age_gender_bkts.country_destination == country].population_in_thousands.values))
    pops.append(sum(data_age_gender_bkts[data_age_gender_bkts.country_destination == country].population_in_thousands.values))

data_countries["population_in_thousands"] = pops


country: AU, popul: 23923
country: CA, popul: 35871
country: DE, popul: 82562
country: ES, popul: 47203
country: FR, popul: 64983
country: GB, popul: 63840
country: IT, popul: 61145
country: NL, popul: 16848
country: PT, popul: 10609
country: US, popul: 325132


In [62]:
# set avg_monthly_google_search_volumes
avg_monthly_search_volumes = {"AU": 301000, "CA": 246000, "DE": 246000, "ES": 165000, "FR": 201000, "GB": 150000, "IT": 201000, "NL": 110000, "PT": 110000, "US": 500000}

data_countries["avg_monthly_search_volume"] = data_countries.apply(lambda x: avg_monthly_search_volumes[x.country_destination], axis=1)
#for country, volume in avg_monthly_search_volumes.items():
#    data_countries[data_countries.country_destination==country]["avg_monthly_search_volume"] = volume

In [67]:
data_countries

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance,population_in_thousands,avg_monthly_search_volumes,avg_monthly_search_volume,distance_km_norm,destination_km2_norm,language_levenshtein_distance_norm,population_in_thousands_norm,avg_monthly_search_volume_norm
0,AU,-26.853388,133.275160,15297.7440,7741220,eng,0.00,23923,0,301000,1.000000,0.774372,0.000000,0.042331,0.489744
1,CA,62.393303,-96.818146,2828.1333,9984670,eng,0.00,35871,0,246000,0.184873,1.000000,0.000000,0.080318,0.348718
2,DE,51.165707,10.452764,7879.5680,357022,deu,72.61,82562,0,246000,0.515080,0.031728,0.760712,0.228769,0.348718
3,ES,39.896027,-2.487694,7730.7240,505370,spa,92.25,47203,0,165000,0.505351,0.046648,0.966475,0.116348,0.141026
4,FR,46.232193,2.209667,7682.9450,643801,fra,92.06,64983,0,201000,0.502227,0.060570,0.964484,0.172878,0.233333
5,GB,54.633220,-3.432277,6883.6590,243610,eng,0.00,63840,0,150000,0.449979,0.020322,0.000000,0.169244,0.102564
6,IT,41.873990,12.564167,8636.6310,301340,ita,89.40,61145,0,201000,0.564569,0.026128,0.936616,0.160675,0.233333
7,NL,52.133057,5.295250,7524.3203,41543,nld,63.22,16848,0,110000,0.491858,0.000000,0.662336,0.019836,0.000000
8,PT,39.553444,-7.839319,7355.2534,92090,por,95.45,10609,0,110000,0.480806,0.005084,1.000000,0.000000,0.000000
9,US,36.966427,-95.844030,0.0000,9826675,eng,0.00,325132,0,500000,0.000000,0.984110,0.000000,1.000000,1.000000


In [65]:
# normalize features
from sklearn import preprocessing
norm_features = ["distance_km", "destination_km2", "language_levenshtein_distance", "population_in_thousands", "avg_monthly_search_volume"]
min_max_scaler = preprocessing.MinMaxScaler()

for feat in norm_features:
    x = data_countries[feat].values.astype(float)
    x_scaled = min_max_scaler.fit_transform(x.reshape(-1,1))
    data_countries[feat+"_norm"] = x_scaled

# Optionally add static normalized country features to df_all ?!

In [70]:
for country in data_countries.country_destination.values:
    #print "country: %s, language_levenshtein_distance: %s" % (country, [X_countries[X_countries.country_destination==country].language_levenshtein_distance.values])
    df_all["language_levenshtein_distance_norm_%s"%country] = np.array([data_countries[data_countries.country_destination==country].language_levenshtein_distance_norm.values] * len(df_all))
    df_all["distance_km_norm_%s"%country] = np.array([data_countries[data_countries.country_destination==country].distance_km_norm.values] * len(df_all))
    df_all["destination_km2_norm_%s"%country] = np.array([data_countries[data_countries.country_destination==country].destination_km2_norm.values] * len(df_all))
    df_all["population_in_thousands_norm_%s"%country] = np.array([data_countries[data_countries.country_destination==country].population_in_thousands_norm.values] * len(df_all))
    df_all["avg_monthly_search_volume_norm_%s"%country] = np.array([data_countries[data_countries.country_destination==country].avg_monthly_search_volume_norm.values] * len(df_all))
    

## Set age_bucket_pop_perc_ features [takes too much time!!!]

In [72]:
def get_age_bucket_pop(row):
    if row.age == 0:
        #print "age is zero, returning.."
        return 0
    if row.age > 99:
        age_bucket = "100+"
    else:
        age_bucket = "%d-%d" % (row.age - row.age%5, row.age + (4-row.age%5))
    #row.age_group = "%d-%d" % (row.age - row.age%5, row.age + (4-row.age%5))
    #print "age is %d, setting it to %s" % (row.age, row.age_group)
    #age_bucket_pop = sum(data_age_gender_bkts[(data_age_gender_bkts.age_bucket==age_bucket) & (data_age_gender_bkts.country_destination==country)].population_in_thousands)
    return sum(data_age_gender_bkts[(data_age_gender_bkts.age_bucket==age_bucket) & (data_age_gender_bkts.country_destination==country)].population_in_thousands)
    #country_pop = X_countries[X_countries.country_destination==country].population_in_thousands
    #return age_bucket_pop/float(country_pop)
    #return age_bucket_pop

for country in data_countries.country_destination.unique():
    country_pop = data_countries[data_countries.country_destination==country].population_in_thousands
    df_all["age_bucket_pop_perc_%s"%country] = df_all.apply(get_age_bucket_pop, axis=1)/float(country_pop)

#print "processed x_t:\n",x_t

In [14]:
df_all.tail()

NameError: name 'df_all' is not defined

In [76]:
df_all.to_csv('./data/df_all_no_sess.csv', index=False)
#y.to_csv('./data/y.csv', index=False)

In [2]:
df_all.shape

NameError: name 'df_all' is not defined